In [ ]:
!pip install -q bertopic sentence-transformers hdbscan

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 36.9 MB/s eta 0:00:00


In [ ]:
import torch
if torch.cuda.is_available():
    device = "cuda"
    print("GPU is enabled.")
else:
    device = "cpu"
    print("GPU not detected, using CPU.")

GPU is enabled.


In [ ]:
import pandas as pd
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
from hdbscan import HDBSCAN
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/data_analysis.csv')
df = df[df['cleaned_text'].notnull()].reset_index(drop=True)

In [ ]:
embedding_model = SentenceTransformer("all-mpnet-base-v2", device=device)  # Accurate + GPU support
docs = df['cleaned_text'].tolist()
embeddings = embedding_model.encode(docs, show_progress_bar=True, device=device)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/298 [00:00<?, ?it/s]

In [ ]:
vectorizer_model = CountVectorizer(
    stop_words="english", # removed ngrams/ bigrams
    min_df=1,  # filter very rare words, using default because higher values tend to mess up
    max_features=5000 #chaing it from default to 5000
)

In [ ]:
hdbscan_model = HDBSCAN(min_cluster_size=5, min_samples=5, prediction_data=True)

In [ ]:
topic_model = BERTopic(
    embedding_model=embedding_model,
    vectorizer_model=vectorizer_model,
    hdbscan_model=hdbscan_model,
    calculate_probabilities=True,
    verbose=True
)

In [ ]:
topics, probs = topic_model.fit_transform(docs, embeddings)

2025-04-21 10:16:28,170 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-04-21 10:17:15,549 - BERTopic - Dimensionality - Completed ✓
2025-04-21 10:17:15,551 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-04-21 10:17:40,095 - BERTopic - Cluster - Completed ✓
2025-04-21 10:17:40,115 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-04-21 10:17:42,856 - BERTopic - Representation - Completed ✓


In [ ]:
df['topic_id'] = topics

In [ ]:
df_filtered = df[df['topic_id'] != -1].reset_index(drop=True)

In [ ]:
# new try
bar_chart = topic_model.visualize_barchart(top_n_topics=10)
bar_chart.show()

In [ ]:
fig = topic_model.visualize_distribution(probs[0], min_probability=0.01)
fig.show()

In [ ]:
named_topics = topic_model.get_topic_info()
df_named = df_filtered.merge(named_topics, how="left", left_on="topic_id", right_on="Topic")

In [ ]:
print(f"Number of real topics (excluding outliers): {named_topics[named_topics['Topic'] != -1].shape[0]}")


Number of real topics (excluding outliers): 224


In [ ]:
def show_top_docs_per_topic(topic_num, n=5):
    sample = df_named[df_named['topic_id'] == topic_num][['title', 'cleaned_text']].head(n)
    print(f"\n--- Top {n} Documents for Topic {topic_num} ---")
    for i, row in sample.iterrows():
        print(f"\nTitle: {row['title']}\nText: {row['cleaned_text'][:500]}\n")

In [ ]:
show_top_docs_per_topic(6, 3)


--- Top 3 Documents for Topic 6 ---

Title: Stop-word elimination and stemmer in python
Text: stop word elimination stemmer python p somewhat large document want stop word elimination stemming words document python anyone know shelf package code fast enough large documents also welcome thanks p p href http www nltk org rel noreferrer nltk supports p


Title: Stop word removal in Javascript
Text: stop word removal javascript p hi looking library remove stop words text code javascript code end goal calculate tf idf convert given document vector space code javascript code anyone point library help library remove stop words would also great p p think libraries thing need download words href https www ranks nl stopwords rel nofollow noreferrer https www ranks nl stopwords p p replace words follows p pre code text text replace stopword quot quot code pre


Title: Package tm stop-word parameter
Text: package tm stop word parameter p trying filter stop words following documents using package 

In [ ]:
# Display the topic names and top words
named_topics = topic_model.get_topic_info()
print("=== Topic Summary ===")
print(named_topics[['Topic', 'Name', 'Count']].head(12))  # Adjust .head() as needed to show more topics


=== Topic Summary ===
    Topic                                              Name  Count
0      -1                             -1_code_word_text_pre   3212
1       0                             0_bert_model_tf_torch   1191
2       1                           1_install_spacy_pip_lib    271
3       2                         2_module_pip_install_nltk    133
4       3                3_matcher_pattern_matches_match_id    127
5       4                      4_125_sklearn_x_train_x_test    117
6       5                              5_tl_frequency_hl_fw    107
7       6                6_stopwords_stop_stop_words_remove    105
8       7   7_classifier_classify_naivebayesclassifier_true    103
9       8                             8_xd8_xd9_utf_unicode     99
10      9  9_countvectorizer_vectorizer_idf_tfidfvectorizer     96
11     10                          10_list_replace_key_item     76


In [ ]:
# total topics from the model
topic_model.get_topic_info()


,Topic,Count,Name,Representation,Representative_Docs
0,-1,3212,-1_code_word_text_pre,"[code, word, text, pre, words, nltk, quot, gt,...",[extracting words using nltk truncate last 39 ...
1,0,1191,0_bert_model_tf_torch,"[bert, model, tf, torch, keras, transformers, ...",[transfer learning distillation model loss dec...
2,1,271,1_install_spacy_pip_lib,"[install, spacy, pip, lib, blis, python3, pack...",[error message installing spacy 2 3 5 quot mic...
3,2,133,2_module_pip_install_nltk,"[module, pip, install, nltk, python, py, lib, ...",[python version import confusion p trying get ...
4,3,127,3_matcher_pattern_matches_match_id,"[matcher, pattern, matches, match_id, span, op...",[spacy matcher return rule patterns p need hel...
...,...,...,...,...,...
220,219,5,219_freqdist_webtext_letters_sample,"[freqdist, webtext, letters, sample, bar, oper...",[typeerror 39 map 39 object subscriptable nltk...
221,220,5,220_excel_xlsx_text_field_det,"[excel, xlsx, text_field, det, read_excel, tex...",[writing spacy python tokens excel file p try ...
222,221,5,221_models_spacy_io_usage,"[models, spacy, io, usage, explosion, train, j...",[spacy api version web version results differe...
223,222,5,222_pron_noun_verb_ii,"[pron, noun, verb, ii, gender, alice, df, lite...",[remove square brackets result pos_tag p want ...


In [ ]:
#filtering confidence by threshold
confidence_threshold = 0.001
df_results = pd.DataFrame({
    'document': docs,
    'topic': topics,
    'confidence': probs[:,0]
})
df_filtered = df_results[df_results['confidence'] >= confidence_threshold]
df_filtered = df_filtered[df_filtered['topic'] != -1]
print(f"Original documents: {len(df_results)}")
print(f"Filtered documents: {len(df_filtered)}")

Original documents: 9517
Filtered documents: 3509


In [ ]:
#filtered topic names
named_topics = topic_model.get_topic_info()
df_filtered_named = df_filtered.merge(named_topics, how='left', left_on='topic', right_on='Topic')


In [ ]:
# top 10 posts per topic (trying 15 posts, if not working change it back to 15)
top_docs_per_topic = (
    df_filtered_named
    .sort_values(by='confidence', ascending=False)
    .groupby('topic')
    .head(15)
    .reset_index(drop=True)
)
top_docs_per_topic[['topic', 'Name', 'confidence', 'document']].head(20)  # Preview


,topic,Name,confidence,document
0,0,0_bert_model_tf_torch,1.000000,train bert model scratch task specific archite...
1,0,0_bert_model_tf_torch,1.000000,encode list sentences embeddings using hugging...
2,0,0_bert_model_tf_torch,1.000000,attributeerror 39 trainingarguments 39 object ...
3,0,0_bert_model_tf_torch,1.000000,fix error oserror lt model gt appear file name...
4,0,0_bert_model_tf_torch,1.000000,underfitting pre trained glove lstm model accu...
5,0,0_bert_model_tf_torch,1.000000,get words output xlnet using transformers libr...
6,0,0_bert_model_tf_torch,1.000000,incompatible shapes 128 37 vs 128 34 p added a...
7,0,0_bert_model_tf_torch,1.000000,many one setting lstm using cntk p want many o...
8,0,0_bert_model_tf_torch,1.000000,sequence sequence modeling python p trying mak...
9,0,0_bert_model_tf_torch,1.000000,huggingface pretrained model 39 tokenizer mode...


In [ ]:
topic_model.visualize_topics()

In [ ]:
df_filtered = df_filtered.reset_index(drop=True)

#Merge back with original data
df_filtered_full = df_filtered.join(df)

df_filtered_named = df_filtered_full.merge(named_topics, how='left', left_on='topic', right_on='Topic')


In [ ]:
# checking how many topics were there after thresholding
df_filtered_named['topic'].nunique()
df_filtered_named['topic'].value_counts()


,count
topic,
0,1191
1,211
2,115
4,96
7,93
...,...
202,1
125,1
210,1


In [ ]:
with pd.ExcelWriter("bertopic_enriched_output_15posts.xlsx") as writer:
    df_filtered_named.to_excel(writer, sheet_name="All_Filtered", index=False)
    top_docs_per_topic.to_excel(writer, sheet_name="Top_10_Per_Topic", index=False)


### checking top 10 for filtered topics, so that the best matched categories alone are considered

In [ ]:
show_top_docs_per_topic(6, 15)


--- Top 15 Documents for Topic 6 ---

Title: Stop-word elimination and stemmer in python
Text: stop word elimination stemmer python p somewhat large document want stop word elimination stemming words document python anyone know shelf package code fast enough large documents also welcome thanks p p href http www nltk org rel noreferrer nltk supports p


Title: Stop word removal in Javascript
Text: stop word removal javascript p hi looking library remove stop words text code javascript code end goal calculate tf idf convert given document vector space code javascript code anyone point library help library remove stop words would also great p p think libraries thing need download words href https www ranks nl stopwords rel nofollow noreferrer https www ranks nl stopwords p p replace words follows p pre code text text replace stopword quot quot code pre


Title: Package tm stop-word parameter
Text: package tm stop word parameter p trying filter stop words following documents using package

note: topic 6 very well resembles stop word related issues

In [ ]:
show_top_docs_per_topic(21, 15)


--- Top 15 Documents for Topic 21 ---

Title: Algorithm to determine how positive or negative a statement/text is
Text: algorithm determine positive negative statement text p need algorithm determine sentence paragraph article negative positive tone better yet negative positive p p instance p blockquote blockquote p jason worst user ever witnessed 10 p p jason user 0 p p jason best user ever seen 10 p p jason best sucking 10 p p okay jason worst bad 10 p blockquote blockquote p easy huh p p expect somebody explain algorithm assume already much work something like academia somewhere point articles research would l


Title: NLP: Qualitatively &quot;positive&quot; vs &quot;negative&quot; sentence
Text: nlp qualitatively quot positive quot vs quot negative quot sentence p need help determining best approach analyzing industry specific sentences e movie reviews positive vs negative seen libraries opennlp low level gives basic sentence composition need higher level structure hopefully wordl

note: topic 21 well matches sentiment analysis category

In [ ]:
show_top_docs_per_topic(20, 14)


--- Top 14 Documents for Topic 20 ---

Title: Classifying Text Based on Groups of Keywords?
Text: classifying text based groups keywords p list requirements software project assembled remains predecessor requirement map one categories categories consists group keywords trying find algorithm would give score ranking categories requirement likely fall results would use starting point categorize requirements p p example suppose requirement p blockquote p system shall apply deposits customer specified account p blockquote p categories keywords p ol li customer transactions deposits deposit custo


Title: I want a machine to learn to categorize short texts
Text: want machine learn categorize short texts p ton short stories 500 words long want categorize one let say 20 categories p ul li entertainment li li food li li music li li etc li ul p hand classify bunch want implement machine learning guess categories eventually best way approach standard approach machine learning using think decisi

note: topic 20 - classification using only 14 posts

In [ ]:
show_top_docs_per_topic(61, 16)


--- Top 16 Documents for Topic 61 ---

Title: What is the best &quot;turnkey&quot; stemming algorithm?
Text: best quot turnkey quot stemming algorithm p need good stemming algorithm project working suggested look porter stemmer checked page porter stemmer found deprecated favor snowball stemmer p p need good stemmer really spend significant time implementing optimizing best shelf freely available stemmer non free stemmers available reasonable price snowball stemmer best bet p p href http snowball tartarus org algorithms english stemmer html rel noreferrer porter2 stemmer one decided go seemed porter st


Title: Stemming - code examples or open source projects?
Text: stemming code examples open source projects p stemming something needed tagging systems use delicious time manage prune tags bit careful blog perfect write software embedded systems would much functional helpful user included stemming p p instance br parse br parser br parsing p p mean thing whatever system putting p p ide

note: topic 61 - well represents stemming totalling 16 posts

In [ ]:
show_top_docs_per_topic(9, 69)


--- Top 69 Documents for Topic 9 ---

Title: Is there a way to convert nltk featuresets into a scipy.sparse array?
Text: way convert nltk featuresets scipy sparse array p trying use scikit learn needs numpy scipy arrays input featureset generated nltk consists unigram bigram frequencies could manually lot effort wondering solution overlooked p p know note scikit learn em n em gram frequency counting assuming word level em n em grams p pre code sklearn feature_extraction text import countvectorizer wordngramanalyzer v countvectorizer analyzer wordngramanalyzer min_n 1 max_n 2 x v fit_transform files code pre p code


Title: efficient Term Document Matrix with NLTK
Text: efficient term document matrix nltk p trying create term document matrix nltk pandas wrote following function p pre code def fndtm_corpus xcorpus import pandas pd create term document matrix nltk corpus fd_list x range 0 len xcorpus fileids fd_list append nltk freqdist xcorpus words xcorpus fileids x dtm pd dataframe fd

note: all 69 fine for topic 9 - topic name encoding strategies

In [ ]:
show_top_docs_per_topic(24, 41)


--- Top 41 Documents for Topic 24 ---

Title: Named Entity Recognition Libraries for Java
Text: named entity recognition libraries java p looking simple good enough named entity recognition library dictionary java looking process emails documents extract basic information like names places address dates p p looking around seems heavy side full nlp kind projects p p recommendations p p btw recently ran across href http www opencalais com rel nofollow noreferrer opencalais seems havethe functionality looking p


Title: Should I use LingPipe or NLTK for extracting names and places?
Text: use lingpipe nltk extracting names places p looking extract names places short bursts text example p pre cardinals vs jays toronto daniel nestor nenad zimonjic play jonas bjorkman w kevin ullyett paris time announced jenson button pole position brawn mercedes monaco pre p data currently mysql database pretty much separate record athlete though names sometimes spelled wrong etc p p would like extract athl

note: NER topic 24, choosing top 40

In [ ]:
show_top_docs_per_topic(70, 15)


--- Top 15 Documents for Topic 70 ---

Title: Clustering text in Python
Text: clustering text python p need cluster text documents researching various options looks like lingpipe cluster plain text without prior conversion vector space etc tool seen explicitly claims work strings p p python tools cluster text directly best way handle p p quality text clustering depends mainly two factors p ol li p notion similarity documents want cluster example easy distinguish newsarticles sports politics vector space via tfidf cosine distance lot harder cluster product reviews good bad


Title: k-fold Cross Validation for determining k in k-means?
Text: k fold cross validation determining k k means p document clustering process data pre processing step first applied singular vector decomposition obtain code u code code code code vt code choosing suitable number eigen values truncated code vt code gives good document document correlation read href http en wikipedia org wiki latent_semantic_analysis 

note: topic 70 - clustering total 15 posts

In [ ]:
show_top_docs_per_topic(89, 17)


--- Top 17 Documents for Topic 89 ---

Title: Compare many text files that contain duplicate &quot;stubs&quot; from the previous and next file and remove duplicate text automatically
Text: compare many text files contain duplicate quot stubs quot previous next file remove duplicate text automatically p large number text files 1000 containing article academic journal unfortunately article file also contains stub end previous article beginning beginning next article end p p need remove stubs preparation running frequency analysis articles stubs constitute duplicate data p p simple field marks beginning end article cases however duplicate text seem formatted line cases p p script com


Title: The lines that stand out in a file, but aren&#39;t exact duplicates
Text: lines stand file 39 exact duplicates p combing webapp log file statements stand p p lines similar uninteresting pass unix code uniq code however filters nothing lines slightly different different timestamp similar statements m

note: above name - similarity

In [ ]:
show_top_docs_per_topic(28, 41)


--- Top 41 Documents for Topic 28 ---

Title: A StringToken Parser which gives Google Search style &quot;Did you mean:&quot; Suggestions
Text: stringtoken parser gives google search style quot mean quot suggestions h2 seeking method h2 h2 take whitespace separated tokens string return suggested word h2 p br strong ie strong br google search take em fonetic wrd nterpreterr em br atop result page shows em mean phonetic word interpreter em p p em solution c languages java would preferred em p p br strong existing open libraries perform functionality strong p p strong way utilise google api request suggested word strong p p article href ht


Title: Looking for Java spell checker library
Text: looking java spell checker library p looking open source java spell checking library dictionaries least following languages french german spanish czech suggestion p p check href http jazzy sourceforge net rel noreferrer jazzy used high profile java applications two problems p ol li updated since 2005

note: spell checker

In [ ]:
show_top_docs_per_topic(78, 20)


--- Top 20 Documents for Topic 78 ---

Title: How to automatically excerpt user generated content?
Text: automatically excerpt user generated content p run website allows users write blog post would really like summarize written content use fill code lt meta name description gt code tag example p p em methods employ automatically summarize describe contents user generated content br preferably free methods solved problem em p p seen websites copy first 100 words strikes sub optimal solution p p think task summarization challenge select important sentences document p p method described href http www


Title: What are the available tools to summarize or simplify text?
Text: available tools summarize simplify text p library preferably python least open source summarize simplify natural language text p p sure currently libraries text summarization least strong understandable strong text summarization something easily accomplished simple plug amp play library p p links managed find regardi

note: Summarization

In [ ]:
topic_model.save("bertopic_model", save_embedding_model=True)

2025-04-21 10:25:00,915 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.
